In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/Users/duanyihong/Desktop/financial markets/Assignment 7')

In [3]:
BLport=pd.DataFrame(pd.read_excel('BL_port.xlsx'))
Cov=pd.read_csv('Cov.csv')

## Derive the optimal portfolio weights

In [4]:
#Clean the data and transform it to a Covariance matrix
Cov_val=Cov[['Australia','Canada','France','Germany','Japan','UK','USA']]
Cov_val.index=Cov_val.columns

In [35]:
#Calculate the inverse of covariance matrix
Cov_inv=pd.DataFrame(np.linalg.pinv(Cov_val.values),Cov_val.columns,Cov_val.index)
#Set risk aversion as 2.5
result=2.5*np.dot(Cov_val,BLport['Equilibrium Portfolio Weight (%)']).round(decimals=2)

In [36]:
BLport['Equilibrium expected return(%)']=result
BLport

,Country,Equity Index Volatility (%),Equilibrium Portfolio Weight (%),Equilibrium expected return(%),R_BL,BL_weight/Original%,BL_weight/Updated%
0,Australia,16.0,1.6,3.950,4.2,1.6,1.6
1,Canada,20.3,2.2,6.925,7.4,3.3,3.3
2,France,24.8,5.2,8.350,9.0,-1.0,-9.6
3,Germany,27.1,5.5,9.025,10.4,26.0,54.7
4,Japan,21.0,11.6,4.300,4.4,11.2,11.2
5,UK,20.0,12.4,6.775,6.9,-1.8,-21.9
6,USA,18.7,61.5,7.575,7.9,60.5,60.5


## Use CAPM to derive the optimal weights

In [37]:
#Calculate the betas for each country with respect to USA
CAPM=pd.DataFrame(columns=['Beta','Expected return%'],index=Cov_val.index)
#The USA equity risk premium is about 5% in 2019
#Risk-free rate is about 2.5%
for i in Cov_val.index:
    CAPM['Beta'].loc[i]=Cov_val['USA'].loc[i]/Cov_val['USA'].loc['USA']
    CAPM['Expected return%']=(CAPM['Beta']*0.05+0.025)*100

In [38]:
CAPM['Expected return%']

Australia        4.6
Canada       6.72857
France       6.92857
Germany      7.22857
Japan        4.21429
UK           5.98571
USA              7.5
Name: Expected return%, dtype: object

## Add a view and derive new estimates of the risk premia

In [39]:
#first form P matrix
#my view: Japan outperforms all the European countries by 5%
#P_prime=[0,0,-0.3,-0.4,1,-0.3,0]
#test the case in class that Germany will outperform the rest of Europe
P_prime=[0,0,-0.3,1,0,-0.7,0]
Q=5
#suppose my view has variance of 0.2^2
myvar=0.04
#set tao as 1: I'm not that confident about my result
tao=1
addi=np.dot(np.linalg.pinv(tao*Cov_val),BLport['Equilibrium expected return(%)'])+(1/myvar)*np.transpose(P_prime)*Q
Cov_new=np.linalg.pinv(np.linalg.pinv(tao*Cov_val)+(1/myvar)*np.outer(np.transpose(P_prime),P_prime))

In [40]:
BL_return=np.dot(Cov_new,addi).round(decimals=1)

In [41]:
BLport['R_BL']=BL_return

In [42]:
BLport['BL_weight/Original%']=np.dot(np.linalg.pinv(2.5*Cov_val),BL_return).round(decimals=1)

In [43]:
BLport['BL_weight/Updated%']=np.dot(np.linalg.pinv(2.5*Cov_new),BL_return).round(decimals=1)

In [44]:
BLport

,Country,Equity Index Volatility (%),Equilibrium Portfolio Weight (%),Equilibrium expected return(%),R_BL,BL_weight/Original%,BL_weight/Updated%
0,Australia,16.0,1.6,3.950,4.2,1.6,1.6
1,Canada,20.3,2.2,6.925,7.4,3.3,3.3
2,France,24.8,5.2,8.350,9.0,-1.0,-9.6
3,Germany,27.1,5.5,9.025,10.4,26.0,54.7
4,Japan,21.0,11.6,4.300,4.4,11.2,11.2
5,UK,20.0,12.4,6.775,6.9,-1.8,-21.9
6,USA,18.7,61.5,7.575,7.9,60.5,60.5
